# 🔬 Bose-Hubbard Model: Mott Insulator to Superfluid

Explore the quantum phase transition in the Bose-Hubbard model between Mott insulator and superfluid phases.

In [ ]:
!rm -rf PyTenNet && git clone https://github.com/tigantic/PyTenNet.git
import sys; sys.path.insert(0, 'PyTenNet')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tensornet import MPS, bose_hubbard_mpo, dmrg

torch.manual_seed(42)
print(f'PyTorch {torch.__version__}')

## 1. Ground State Energy vs U/t

In [ ]:
# H = -t Σ(b†_i b_j + h.c.) + (U/2) Σ n_i(n_i-1) - μ Σ n_i
# Phase transition at (U/t)_c ≈ 3.4 for unit filling in 1D

L = 8  # Small system due to larger local dimension
n_max = 3  # Max bosons per site (d = n_max + 1 = 4)
t = 1.0
mu = 0.5  # Chemical potential

U_values = [0.5, 1.0, 2.0, 3.0, 4.0, 6.0, 8.0, 10.0]
energies = []

for U in U_values:
    H = bose_hubbard_mpo(L=L, n_max=n_max, t=t, U=U, mu=mu)
    psi = MPS.random(L=L, d=n_max+1, chi=32)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=32)
    energies.append(E)
    print(f'U/t = {U/t:4.1f}: E = {E:.6f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot([U/t for U in U_values], energies, 'o-', linewidth=2, markersize=8, color='darkblue')
plt.axvline(x=3.4, color='red', linestyle='--', linewidth=2, label='Critical (U/t)_c ≈ 3.4')
plt.fill_betweenx(plt.ylim(), 0, 3.4, alpha=0.1, color='blue', label='Superfluid')
plt.fill_betweenx(plt.ylim(), 3.4, 12, alpha=0.1, color='orange', label='Mott insulator')
plt.xlabel('U/t', fontsize=12)
plt.ylabel('Ground state energy', fontsize=12)
plt.title(f'Bose-Hubbard Model (L={L}, n_max={n_max})', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Particle Number Variance (Order Parameter)

In [ ]:
# Number operator: n|m> = m|m>
def number_operator(n_max):
    d = n_max + 1
    n = torch.zeros(d, d, dtype=torch.float64)
    for m in range(d):
        n[m, m] = m
    return n

def number_squared_operator(n_max):
    d = n_max + 1
    n2 = torch.zeros(d, d, dtype=torch.float64)
    for m in range(d):
        n2[m, m] = m * m
    return n2

n_op = number_operator(n_max)
n2_op = number_squared_operator(n_max)

print('Number operator:')
print(n_op)

In [ ]:
# Compute variance Var(n) = <n²> - <n>² at each site, then average
L = 8
n_max = 3
U_values = np.linspace(0.5, 10.0, 15)
variances = []

for U in U_values:
    H = bose_hubbard_mpo(L=L, n_max=n_max, t=1.0, U=U, mu=0.5)
    psi = MPS.random(L=L, d=n_max+1, chi=32)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=32)
    
    # Compute variance at central site
    site = L // 2
    psi_copy = psi.copy()
    n_expect = psi_copy.expectation_local(n_op, site).item()
    psi_copy = psi.copy()
    n2_expect = psi_copy.expectation_local(n2_op, site).item()
    var_n = n2_expect - n_expect**2
    variances.append(var_n)
    print(f'U/t = {U:.1f}: <n> = {n_expect:.3f}, Var(n) = {var_n:.4f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(U_values, variances, 'o-', linewidth=2, markersize=8, color='green')
plt.axvline(x=3.4, color='red', linestyle='--', linewidth=2, label='Critical (U/t)_c ≈ 3.4')
plt.xlabel('U/t', fontsize=12)
plt.ylabel('Particle number variance Var(n)', fontsize=12)
plt.title('Order Parameter: Number Fluctuations', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Superfluid: large variance (delocalized particles)')
print('Mott insulator: small variance (localized particles)')

## 3. Entanglement in Both Phases

In [ ]:
L = 10
n_max = 3

# Compare superfluid (U=1) vs Mott insulator (U=8)
U_compare = [1.0, 8.0]
labels = ['U/t=1 (Superfluid)', 'U/t=8 (Mott)']
colors = ['blue', 'orange']

plt.figure(figsize=(10, 6))

for U, label, color in zip(U_compare, labels, colors):
    H = bose_hubbard_mpo(L=L, n_max=n_max, t=1.0, U=U, mu=0.5)
    psi = MPS.random(L=L, d=n_max+1, chi=32)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=32)
    
    entropies = [psi.entropy(bond).item() for bond in range(L-1)]
    plt.plot(range(1, L), entropies, 'o-', label=label, color=color, linewidth=2)
    print(f'{label}: E = {E:.4f}, S_max = {max(entropies):.4f}')

plt.xlabel('Bond position', fontsize=12)
plt.ylabel('Entanglement entropy', fontsize=12)
plt.title('Entanglement: Superfluid vs Mott Insulator', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Density Profile

In [ ]:
L = 10
n_max = 3

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, U in enumerate([1.0, 8.0]):
    H = bose_hubbard_mpo(L=L, n_max=n_max, t=1.0, U=U, mu=0.5)
    psi = MPS.random(L=L, d=n_max+1, chi=32)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=32)
    
    densities = []
    for site in range(L):
        psi_copy = psi.copy()
        n_expect = psi_copy.expectation_local(n_op, site).item()
        densities.append(n_expect)
    
    ax = axes[idx]
    phase_name = 'Superfluid' if U == 1 else 'Mott'
    ax.bar(range(L), densities, color='steelblue' if U==1 else 'coral', alpha=0.8)
    ax.set_xlabel('Site', fontsize=12)
    ax.set_ylabel('<n>', fontsize=12)
    ax.set_title(f'U/t = {U} ({phase_name})', fontsize=14)
    ax.set_ylim(0, max(densities) * 1.2)
    ax.grid(True, alpha=0.3, axis='y')
    print(f'U/t={U}: Mean density = {np.mean(densities):.3f}')

plt.tight_layout()
plt.show()